In [45]:
from PyPDF2 import PdfReader, PdfWriter
import os
from dotenv import load_dotenv
import openai

In [4]:
pdf_file= open(r"C:\Users\ayhan\Desktop\ChefApp\Italian_Italian_200_Recipes_for_Authenti.pdf","rb")

In [18]:
load_dotenv()

True

In [3]:
reader = PdfReader(pdf_file)

In [62]:
RECIPES_FOLDER = "recipes"
FORMATTED_RECIPES_FOLDER= "formatted-recipes"
os.makedirs(r"C:\Users\ayhan\Desktop\ChefApp"+"/recipes",exist_ok=True)
os.makedirs(r"C:\Users\ayhan\Desktop\ChefApp"+"/formatted-recipes",exist_ok=True)

In [8]:
def extract_text_from_pdf(pdf_path):

    reader = PdfReader(open(pdf_path,"rb"))
    
    saved_text= ""
    for index, page in enumerate(reader.pages):
        
        text = page.extract_text()
        saved_text += text
        print(len(saved_text.split()))
        
        if len(saved_text.split()) >= 1000:
            print("ok",index)
            filename = f"recipe{index}.txt"
            file_path = os.path.join(RECIPES_FOLDER,filename)
            with open(file_path,"w",encoding="utf-8") as f:
                f.write(saved_text)
            saved_text= ""

            
        else:
            print("None")
        
    return "EXTRACTION COMPLETED"

pdf_path = r"C:\Users\ayhan\Desktop\ChefApp\Italian_Italian_200_Recipes_for_Authenti.pdf"
extract_text_from_pdf(pdf_path)

7
None
19
None
130
None
142
None
163
None
208
None
225
None
423
None
865
None
1159
ok 9
467
None
962
None
962
None
1644
ok 13
337
None
870
None
1192
ok 16
628
None
1161
ok 18
0
None
263
None
698
None
1166
ok 22
435
None
894
None
1123
ok 25
352
None
909
None
1476
ok 28
236
None
601
None
681
None
990
None
1371
ok 33
376
None
965
None
982
None
1038
ok 37
551
None
1153
ok 39
617
None
1137
ok 41
585
None
1044
ok 43
353
None
804
None
804
None
1514
ok 47
814
None
1029
ok 49
22
None
700
None
858
None
1512
ok 53
367
None
917
None
1312
ok 56
0
None
434
None
839
None
1069
ok 60
441
None
999
None
1359
ok 63
504
None
1067
ok 65
328
None
876
None
1233
ok 68
490
None
490
None
984
None
1447
ok 72
332
None
739
None
1295
ok 75
0
None
608
None
1272
ok 78
190
None
815
None
1178
ok 81
672
None
848
None
1284
ok 84
386
None
902
None
1174
ok 87
524
None
1216
ok 89
201
None
489
None
821
None
1118
ok 93
386
None
789
None
789
None
1187
ok 97
473
None
750
None
1476
ok 100
0
None
577
None
939
None
1294
ok 104
572


'EXTRACTION COMPLETED'

In [33]:
RECIPE_GENERATOR_PROMPT = """
{RAW_RECIPE_TEXT}
for the recipe above, choose the most probable options from the option list, you have to choose one of them no matter what:
cuisine: [Choose one of these: Italian Cuisine, Mexican Cuisine, Chinese Cuisine, ...]meal_type: [Choose one of these: Breakfast, Lunch, Dinner, Appetizers, Snacks, Desserts, Beverages, Baking]
dietary_type:[Choose one of these: "Vegetarian Cooking","Vegan Cooking","Gluten-Free Cooking","Keto Cooking","Paleo Cooking","Low-Carb Cooking","Dairy-Free Cooking","Nut-Free Cooking"]
health_and_wellness_categories = [
    "Heart-Healthy Cooking", "Low-Calorie Cooking", "High-Protein Recipes", 
    "Weight-Loss-Friendly Recipes", "Plant-Based Nutrition", "Gluten-Free Cooking", 
    "Keto Cooking", "Paleo Cooking", "Low-Carb Cooking", 
    "Dairy-Free Cooking", "Nut-Free Cooking"]
cooking_skill_levels:[Choose one of these: Beginner-Friendly Recipes, Intermediate Cooking, Advanced Culinary Techniques]
  nutrition_information:
    calories:  (float)(fill required)
    protein: (float)(fill required)
    carbohydrates:(float)(fill required)
    total_fat: 15.0 (float)(fill required)
  ingredients:
    
ingredient_name: //(only the name not features)
    quantity: 1.0 (float)
    unit: (fill required)
    preparation: (fill required)...

then you should fill these according to text, if no info found then you should fill with proper values
  prep_time: //(minutes)(fill required)
  cook_time: //(minutes)(fill required)
  total_time: //(minutes)(fill required)
  servings: 4 (integer)

now, using all the information above, you regenerate the recipe. include related metadata, total calory, total_time  and servings at the top

"""

In [44]:
RECIPE_EXTRACTOR_PROMPT = """
{RAW_PAGES}
--------------------------------------
given multiple raw pages containing multiple recipes above, you will re-format the given text to the following format:
RECIPE#1-NAME
-related context(including serving size , ingredients )
- related instructions
- PREP time, cook time, total time(all 3 are REQUIRED, if not stated clearly, calculate or fill properly)
- nutrition info at the end
(END-OF-THE-RECIPE)
RECIPE#2-NAME
-related context(including ingredients)
- related instructions
- PREP time, cook time, total time(all 3 are REQUIRED, if not stated clearly, calculate or fill properly)
- nutrition info at the end
(END-OF-THE-RECIPE)
...
If for some recipes some parts(timing or nutrition info) are missing or divided into other text files , please MARK those recipes as (END-OF-THE-INCOMPLETE-RECIPE)
"""

In [46]:

APIKEY = os.getenv("OPENAI-API-KEY")
openai.api_key = APIKEY

In [60]:
content = RECIPE_EXTRACTOR_PROMPT.replace("RAW_PAGES",context)
response = openai.ChatCompletion.create(
    model= "gpt-3.5-turbo",
    messages = [
    {"role":"system", "content":"you are a chef assistant, being able to calculate & assess cook timings and nutrition info"},
    {"role":"user", "content":content},
    ]
)

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [63]:
def chat(context:str)->str:
    
    content = RECIPE_EXTRACTOR_PROMPT.replace("RAW_PAGES",context)
    response = openai.ChatCompletion.create(
        model= "gpt-3.5-turbo",
        messages = [
        {"role":"system", "content":"you are a chef assistant, being able to calculate & assess cook timings and nutrition info"},
        {"role":"user", "content":content},
        ])
    
    return ['choices'][0]['message']['content']

In [64]:
for page_name in sorted(os.listdir(RECIPES_FOLDER)):
    file_path = os.path.join(RECIPES_FOLDER,page_name)
    formatted_file_path = os.path.join(FORMATTED_RECIPES_FOLDER,page_name)
    with open(file_path,"r") as f:
        context= f.read()
        response = chat(context)
        with open(formatted_file_path,"w") as f:
            f.write(response)
            print("RECIPE FORMATING SUCCESSFULL")

RateLimitError: You exceeded your current quota, please check your plan and billing details.

## we can also generate recipes in the format given inside the recipe.yaml file, syntetically by GPT-3.5 or GPT-4